# Exercise 1 | Inspecting a Bitcoin address

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import json
from pprint import pprint

import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api

## Notebook Setup

This notebook assumes that you already followed the instructions outlined in `README.md` and installed the [GraphSense Python API](https://github.com/graphsense/graphsense-python) library locally.

First, we must setup our notebook and establish a connection to some hosted GraphSense instance.

We connect to a GraphSense instance hosted by [Iknaio](https://www.ikna.io/) and enter our API key in the provided `config.json` configuration file. An API key will be provided during the tutorial. If you would like to get an API key later, drop an email to contact@iknaio.com

## Load host and API key from config

In [21]:
f = open('config.json')
config = json.load(f)
f.close()

## Configure GraphSense client

Now we configure the GraphSense Python library

In [22]:
configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

We can test if our libary is working by retrieving summary statistics on supported ledgers.

In [23]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    pprint(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 6758109756,
                 'no_addresses': 1045200508,
                 'no_blocks': 762641,
                 'no_entities': 469256656,
                 'no_labels': 3303,
                 'no_tagged_addresses': 250851468,
                 'no_txs': 777477873,
                 'timestamp': 1668122552},
                {'name': 'bch',
                 'no_address_relations': 2303432268,
                 'no_addresses': 333745971,
                 'no_blocks': 766073,
                 'no_entities': 150751693,
                 'no_labels': 86,
                 'no_tagged_addresses': 13674144,
                 'no_txs': 363817727,
                 'timestamp': 1668123907},
                {'name': 'ltc',
                 'no_address_relations': 1382523914,
                 'no_addresses': 160527773,
                 'no_blocks': 2366891,
                 'no_entities': 67746829,
                 'no_labels': 66,
  

# Address API

We start by retrieving some summary statistics on the Internet Archive's donation address. The response basically corresponds to the data shown in the property box of the GraphSense dashboard.

In [24]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = addresses_api.AddressesApi(api_client)

    currency = "btc"
    address = '1Archive1n2C579dMsAu3iC6tWzuQJz8dN'
    include_tags = True

    try:
        resp_addr = api_instance.get_address(currency, address)
        pprint(resp_addr)
    except graphsense.ApiException as e:
        print("Exception when calling AddressesApi->get_address: %s\n" % e)

{'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 25794.39},
                             {'code': 'usd', 'value': 25675.73}],
             'value': 145994584},
 'currency': 'btc',
 'entity': 2647117,
 'first_tx': {'height': 256314,
              'timestamp': 1378415426,
              'tx_hash': '04d92601677d62a985310b61a301e74870fa942c8be0648e16b1db23b996a8cd'},
 'in_degree': 6573,
 'last_tx': {'height': 761605,
             'timestamp': 1667506053,
             'tx_hash': 'ac62d21a2290e80ec7eaab7b4e6bd30090c6e02322a3c81f8702dad9e55f526c'},
 'no_incoming_txs': 5002,
 'no_outgoing_txs': 282,
 'out_degree': 300,
 'status': 'dirty',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 6497523.0},
                                    {'code': 'usd', 'value': 6926346.5}],
                    'value': 61810992019},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 6795673.0},
                                 {'code': 'usd'

In [25]:
print(f"Address {resp_addr.address} received {resp_addr.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_addr.in_degree} addresses.")

Address 1Archive1n2C579dMsAu3iC6tWzuQJz8dN received 6497523.0 EUR from 6573 addresses.


# Entities API

Next, we inspect the corresponding entity, which clusters other addresses that are likely conrolled by the owner of that address. The entity ID is contained in the address response.

In [26]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)

    currency = "btc" 
    entity = resp_addr.entity 
    include_tags = True 

    try:
        resp_entity = api_instance.get_entity(currency, entity)
        pprint(resp_entity)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi->get_entity: %s\n" % e)

{'balance': {'fiat_values': [{'code': 'eur', 'value': 25794.39},
                             {'code': 'usd', 'value': 25675.73}],
             'value': 145994584},
 'best_address_tag': {'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
                      'category': 'organization',
                      'confidence': 'service_data',
                      'confidence_level': 50,
                      'currency': 'BTC',
                      'entity': 2647117.0,
                      'is_cluster_definer': True,
                      'label': 'internet archive',
                      'lastmod': 1636675200,
                      'source': 'https://archive.org/donate/cryptocurrency',
                      'tagpack_creator': 'GraphSense Core Team',
                      'tagpack_is_public': True,
                      'tagpack_title': 'GraphSense Demo TagPack',
                      'tagpack_uri': 'demo.yaml'},
 'currency': 'btc',
 'entity': 2647117,
 'first_tx': {'height': 156529,
      

In [27]:
print(f"Entity {resp_entity.entity} likely controls {resp_entity.no_addresses} addresses, " +
      f"which received {resp_entity.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_entity.in_degree} entities.")

Entity 2647117 likely controls 124 addresses, which received 8710121.0 EUR from 5323 entities.


# Entity Neighbors

Finally, we retrieve all neighbor entities receiving funds from our focus entity. Since it is more covenient to work with dataframes, we use the GraphSense Bulk API interface to retrieve a dataframe of receiving neighbor nodes.

In [28]:
import pandas as pd

data = {'entity': [resp_addr.entity]}

entityDF = pd.DataFrame.from_dict(data)

In [29]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = bulk_api.BulkApi(api_client)

    CURRENCY = "btc"
    operation = "list_entity_neighbors"
    body = {'entity': entityDF['entity'].to_list(),
            'direction': 'out',
            'include_labels': True}

    try:
        respDF = pd.read_csv(
                    api_instance.bulk_csv(CURRENCY, operation, body=body,
                                          num_pages=2,
                                          _preload_content=False))
    except graphsense.ApiException as e:
        print("Exception when calling BulkApi->bulk_csv: %s\n" % e)

In [30]:
respDF

,_error,_info,_request_entity,entity_balance_eur,entity_balance_usd,entity_balance_value,entity_best_address_tag_abuse,entity_best_address_tag_address,entity_best_address_tag_category,entity_best_address_tag_confidence,...,entity_total_received_value,entity_total_spent_eur,entity_total_spent_usd,entity_total_spent_value,labels,labels_count,no_txs,value_eur,value_usd,value_value
0,NaN,NaN,2647117,2.391325e+08,2.380325e+08,1353474480699,NaN,1CsBZRX2CpCg2Bk1tDyCCMYoS8WgeSijCA,entity,override,...,5214491875892682,1.224896e+10,1.426612e+10,5213138401411983,"bitcoin-24.com,epay.info,supercluster",3,2,750.20,986.69,953789282
1,NaN,NaN,2647117,3.124023e+04,3.109652e+04,176817698,NaN,1PEoUKNxTZsc5rFSQvQjeTVwDE9vEDCRWm,service,service_data,...,133520860563439,2.256563e+08,3.006194e+08,133520683745741,bitpay.com,1,1,537.80,673.60,200000000
2,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,48610000000,2.411060e+03,3.074920e+03,48610000000,NaN,0,1,2411.06,3074.92,48610000000
3,NaN,NaN,2647117,2.116921e+06,2.107183e+06,11981636212,NaN,3MEa8XTY78bibHhVbM4JxJE77ij5ZGEBpj,exchange,manual_transaction,...,1177806601629286,1.632319e+11,1.883404e+11,1177794619993074,coinbase,1,77,11922.25,13517.07,262142451
4,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,7858798000,8.870120e+03,1.094745e+04,7858798000,NaN,0,3,1078.04,1397.54,3375700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,100000,4.855000e+01,5.482000e+01,100000,NaN,0,1,29.40,35.70,100000
172,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,600000000,2.229065e+05,2.614470e+05,600000000,NaN,0,1,171694.66,203131.97,600000000
173,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,618131591,2.296426e+05,2.693478e+05,618131591,NaN,0,1,277.73,328.77,1000000
174,NaN,NaN,2647117,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,NaN,...,18899997228,4.163480e+06,4.228014e+06,18899997228,NaN,0,1,4077640.50,4097621.25,18899997228


We filter the result and see the relations between known entity IDs. We also see two known receiving entities.

In [33]:
resultDF = respDF[['_request_entity', 'entity_entity', 'labels', 'no_txs']] \
    .rename(columns={'_request_entity': 'src', 'entity_entity': 'dst', 'labels': 'dst_label'}) \
    .dropna()
resultDF

,src,dst,dst_label,no_txs
0,2647117,109577,"bitcoin-24.com,epay.info,supercluster",2
1,2647117,1084055,bitpay.com,1
3,2647117,4635850,coinbase,77
106,2647117,319021677,"kraken,kraken.com",3
